In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!pip install monai
!pip install nibabel
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import nibabel as nib
from monai.transforms import Spacing
from monai.transforms import SpatialPad
from monai.transforms import Resize
from monai.transforms import CropForeground

import time
import torch

In [4]:
!pwd

/content


In [5]:
os.cpu_count()

2

# Load Image and Label

In [6]:
!ls /gdrive/MyDrive/LiTS_sample

archive.zip  image  image_128  mask


## Resample all same size

In [ ]:
!ls $image_data_dir

volume-0.nii   volume-1.nii   volume-2.nii   volume-3.nii   volume-4.nii
volume-10.nii  volume-20.nii  volume-30.nii  volume-40.nii  volume-50.nii
volume-11.nii  volume-21.nii  volume-31.nii  volume-41.nii  volume-5.nii
volume-12.nii  volume-22.nii  volume-32.nii  volume-42.nii  volume-6.nii
volume-13.nii  volume-23.nii  volume-33.nii  volume-43.nii  volume-7.nii
volume-14.nii  volume-24.nii  volume-34.nii  volume-44.nii  volume-8.nii
volume-15.nii  volume-25.nii  volume-35.nii  volume-45.nii  volume-9.nii
volume-16.nii  volume-26.nii  volume-36.nii  volume-46.nii
volume-17.nii  volume-27.nii  volume-37.nii  volume-47.nii
volume-18.nii  volume-28.nii  volume-38.nii  volume-48.nii
volume-19.nii  volume-29.nii  volume-39.nii  volume-49.nii


In [7]:
from multiprocessing import Pool

class Resampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = (np_image>0.5).astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        np.save(os.path.join(target_dir, file_name), np_image)

    def resize(self, np_image): 
        if self.is_label:
            np_image = np_image > 0.5
        resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        # # load npy
        # np_file = np.load(os.path.join(origin_dir, file))
        # load nifti
        np_file = nib.load(os.path.join(origin_dir, file)).get_fdata()
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=32)
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

class MultiChResampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = np_image.astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        
        # For torch order
        np_image = np.flip(np_image.transpose([2,1,0]), axis=(1,2))
        np.save(os.path.join(target_dir, file_name), np_image) 

    def resize(self, np_image): 
        if self.is_label:
            resizer = Resize(self.resample_size, mode='nearest')
        else: 
            resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        # # load npy
        # np_file = np.load(os.path.join(origin_dir, file))
        # load nifti
        np_file = nib.load(os.path.join(origin_dir, file)).get_fdata()
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=os.cpu_count())
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

In [8]:
source_dir = '/gdrive/MyDrive/LiTS_sample/image'
target_dir = '/gdrive/MyDrive/LiTS_sample/image_128'

source_dir_lb = '/gdrive/MyDrive/LiTS_sample/mask'
target_dir_lb = '/gdrive/MyDrive/LiTS_sample/mask_128'

target_shape = [128,128,128]

image_resampler = MultiChResampler(source_dir, target_dir, target_shape)
label_resampler_1 = MultiChResampler(source_dir_lb, target_dir_lb, target_shape, True)

In [ ]:
image_resampler.resample_all()

In [9]:
label_resampler_1.resample_all()


        consumed_time: 305.358029944s
        


In [ ]:
file_names = os.listdir(target_dir)
image_sizes = [np.load(os.path.join(target_dir, file)).shape for file in file_names]
label_sizes = [np.load(os.path.join(target_dir_lb, file)).shape for file in file_names]

In [ ]:
label_sizes

# Test Codes

## Testing One Case

In [ ]:
img = np.load(get_image_path('000251_20190703_chest.npy'))

In [ ]:
cropper = CropForeground()

In [ ]:
cropper.compute_bounding_box(img[np.newaxis,:])

(array([ 0, 58,  0]), array([297, 512, 512]))

In [ ]:
cropper(img).shape

(297, 454, 512)

In [ ]:
import time

In [ ]:
t = time.perf_counter()

resampled_image = resample_all_1mm(img, ct_info.resolution[0])

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')


consumed_time: 1.6059333749872167s



In [ ]:
img.shape

(297, 512, 512)

In [ ]:
resampled_image.shape

(297, 287, 287)

In [ ]:
label = np.load(get_label_path('000251_20190703_chest.npy'))

In [ ]:
resampled_label = resample_all_1mm(label, ct_info.resolution[0])

In [ ]:
adjust_window(resampled_image, [700,100]).astype(np.int32).min()

0

In [ ]:
plot_image_and_label(adjust_window(resampled_image, [700,100]).astype(np.int32), (resampled_label>0).astype(np.uint8))

In [ ]:
from monai.transforms import Rand3DElastic
sample_np_image = dcm_to_numpy(cases[0], dcm_base_dir) # npy 로 변환한 3D 이미지 하나 입니다
sample_label = np.load('/workspace/BoneMeta_new/labels/{}.npy'.format(cases[0]))
elastic_deform = Rand3DElastic(sigma_range=(35,35), magnitude_range=(10,10), prob=1.0, rotate_range=0, shear_range=0)
deformed_image = elastic_deform(sample_np_image)


## Testing All Sizes

In [ ]:
file_names = os.listdir(all_1mm_image_dir)

In [ ]:
image_sizes = [np.load(os.path.join(all_1mm_image_dir, file)).shape for file in file_names]
label_sizes = [np.load(os.path.join(all_1mm_label_dir, file)).shape for file in file_names]

In [ ]:
image_sizes == label_sizes

True

In [ ]:
plot_image_and_label(np.load(os.path.join(all_1mm_image_dir, file_names[5])), np.load(os.path.join(all_1mm_label_dir, file_names[5])))